In [1]:
from scipy.spatial import distance
from imutils import face_utils
import imutils
import dlib
import cv2
import os
import numpy as np ## pip install numpy
import time

In [2]:
import winsound
frequency = 2500 #2500 Hertz
duration = 1000 # 1000 ms = 1 sec

In [3]:
def eye_aspect_ratio(eye):
    A = distance.euclidean(eye[1], eye[5])
    B = distance.euclidean(eye[2], eye[4])
    C = distance.euclidean(eye[0], eye[3])
    ear = (A + B) / (2.0 * C)
    return ear

In [4]:
img_size = 224
thresh = 0.25
frame_check = 20
detect = dlib.get_frontal_face_detector()
predict = dlib.shape_predictor("shape_predictor_68_face_landmarks.dat")

In [5]:
(lStart, lEnd) = face_utils.FACIAL_LANDMARKS_IDXS["left_eye"]
(rStart, rEnd) = face_utils.FACIAL_LANDMARKS_IDXS["right_eye"]

In [8]:
import tensorflow as tf
model = tf.keras.models.load_model('my_model.h5')
cap=cv2.VideoCapture(0)

new_frame_time = 0;
prev_frame_time = 0;

if not cap.isOpened():
    raise IOError('Cannot open webcam')
    
flag=0
counter  = 0
while True:
    ret, frame=cap.read()
    
    eyecascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_eye.xml')
    gray = cv2.cvtColor(frame,cv2.COLOR_BGR2GRAY)
    eye = eyecascade.detectMultiScale(gray,1.1,4)
    for x,y,w,h in eye:
        roi_gray = gray[y:y+h, x:x+w]
        roi_color = frame[y:y+h, x:x+w]
        #cv2.rectangle(frame,(x,y),(x+w,y+h),(0,255,0), 2)
        eyess = eyecascade.detectMultiScale(roi_gray)
        if len(eyess)==0:
            print('eyes not detected')
        else:
            for (ex,ey,ew,eh) in eyess:
                eyes_roi = roi_color[ey:ey+eh,ex:ex+ew]
            else:
                continue 
    
    final_image = cv2.resize(eyes_roi,(img_size,img_size))
    final_image = np.expand_dims(final_image,axis = 0)
    final_image = final_image/255.0
    
    new_frame_time = time.time()
    
    fps = 1/(new_frame_time-prev_frame_time)
    fps = int(fps)
    prev_frame_time = new_frame_time
    fps = str(fps)
    
    cv2.putText(frame, fps, (200, 70), cv2.FONT_HERSHEY_SIMPLEX, 3, (100, 255, 0), 3, cv2.LINE_AA)
    
    Predictions = model.predict(final_image)
    print(Predictions)
    if(Predictions>0):
        status = 'Open Eyes'
        cv2.putText(frame,status,(150,150),cv2.FONT_HERSHEY_SIMPLEX,3,(0,255,0),2)
        
        x1,y1,w1,h1 = 0,0,175,75
        cv2.rectangle(frame,(x1,y1),(x1+w1,y1+h1),(0,0,0),-1)
        cv2.putText(frame,'Active',(x1+int(w1/10),y1+int(h1/2)),cv2.FONT_HERSHEY_SIMPLEX,0.7,(0,255,0),2)
        
        subjects = detect(gray, 0)
        for subject in subjects:
            shape = predict(gray, subject)
            shape = face_utils.shape_to_np(shape)#converting to NumPy Array
            leftEye = shape[lStart:lEnd]
            rightEye = shape[rStart:rEnd]
            leftEAR = eye_aspect_ratio(leftEye)
            rightEAR = eye_aspect_ratio(rightEye)
            ear = (leftEAR + rightEAR) / 2.0
            leftEyeHull = cv2.convexHull(leftEye)
            rightEyeHull = cv2.convexHull(rightEye)
            cv2.drawContours(frame, [leftEyeHull], -1, (0, 255, 0), 1)
            cv2.drawContours(frame, [rightEyeHull], -1, (0, 255, 0), 1)
            if ear < thresh:
                flag += 1
            #print (flag)
                if flag >= frame_check:
                    cv2.putText(frame, "****************ALERT!****************", (10, 30),
                                cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 0, 255), 2)
                    cv2.putText(frame, "****************ALERT!****************", (10,325),
                                cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 0, 255), 2)
            else:
                flag = 0
    else:
        counter = counter + 1 
        status = 'Closed Eyes'
        cv2.putText(frame,status,(150,150),cv2.FONT_HERSHEY_SIMPLEX,3,(0,0,255),2)
        if counter > 5:
            x1,y1,w1,h1 = 0,0,175,75
            cv2.rectangle(frame,(x1,y1),(x1+w1,y1+h1),(0,0,0),-1)
            cv2.putText(frame, "********SLEEP ALERT!**********", (10, 30),
                                cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 0, 255), 2)
            winsound.Beep(frequency,duration)
            counter = 0
    
    cv2.imshow("Frame", frame)
    key = cv2.waitKey(1) & 0xFF
    if key == ord("q"):
        cv2.destroyAllWindows()
        cap.release()
        break

eyes not detected
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: 'arguments' object has no attribute 'posonlyargs'
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: 'arguments' object has no attribute 'posonlyargs'
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
[[-0.15726896]]
[[-0.13836625]]
eyes not detected
eyes not detected
eyes not detected
[[-0.13836625]]
eyes not detected
[[-0.13836625]]
eyes not detected
[[-0.13836625]]
[[-0.27457348]]
[[-0.20545338]]
eyes not detected
eyes not detected
[[-0.22362565]]
eyes not detected
[[-0.22362565]]
[[-0.30902192]]
eyes not detected
eyes not detected
eyes not detected
[[

eyes not detected
[[0.58532196]]
[[0.7613053]]
eyes not detected
[[0.7613053]]
eyes not detected
[[0.7613053]]
[[0.7613053]]
eyes not detected
eyes not detected
[[0.7613053]]
eyes not detected
eyes not detected
eyes not detected
[[-0.23479064]]
eyes not detected
[[-0.23479064]]
eyes not detected
[[-0.23479064]]
eyes not detected
[[-0.23479064]]
[[-0.23479064]]
[[-0.23479064]]
[[-0.23479064]]
[[-0.23479064]]
[[-0.23479064]]
[[-0.23479064]]
[[-0.23479064]]
eyes not detected
[[-0.23479064]]
[[0.10483742]]
eyes not detected
[[0.10483742]]
[[-0.12931088]]
[[-0.07666759]]
[[-0.07666759]]
[[-0.07666759]]
[[-0.07666759]]
eyes not detected
[[-0.21512285]]
[[-0.21512285]]
eyes not detected
[[-0.21512285]]
[[-0.21512285]]
[[-0.21512285]]
[[-0.21512285]]
[[-0.21512285]]
[[-0.21512285]]
[[-0.21512285]]
[[-0.21512285]]
eyes not detected
[[-0.21512285]]
eyes not detected
eyes not detected
[[-0.21512285]]
eyes not detected
[[-0.21512285]]
eyes not detected
eyes not detected
eyes not detected
[[0.80613

In [ ]:
#cv2.VideoCapture(0)